In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import joblib
import re
import nltk
import mysql.connector
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import formatdate

nltk.download('stopwords')

# MySQL Configuration
mysql_db_config = {
    "host": "network-anomaly-db.cromkkyk46eq.ap-south-1.rds.amazonaws.com",
    "user": "admin",
    "password": "Abc$$1234",
    "database": "network_data"
}

# Load the dumped models
classifier = joblib.load('decision_tree_model_optimized.joblib')
cv = joblib.load('count_vectorizer.joblib')

# Function to clean the texts
def preprocess_text(text):
    log = re.sub('[^a-zA-Z0-9]', ' ', text)
    log = log.lower()
    log = log.split()
    ps = PorterStemmer()
    log = [ps.stem(word) for word in log if not word in set(stopwords.words('english'))]
    log = ' '.join(log)
    return log

def read_data_in_db(table, record_id):
    connection = mysql.connector.connect(**mysql_db_config)
    cursor = connection.cursor()
    
    read_query = f"SELECT info, ipaddress FROM {table} WHERE id = %s"
    cursor.execute(read_query, (record_id,))
    
    row = cursor.fetchone()
    if row is not None:
        log_info = row[0]
        ip_address = row[1]
    else:
        log_info = None
        ip_address = None
    
    cursor.close()
    connection.close()
    
    return log_info, ip_address

def determine_attack_type(info):
    if 'SYN' in info and 'ACK' not in info:
        return "SYN Flood (DoS Attack)"
    if 'ACK' in info and 'PSH' not in info and 'SYN' not in info:
        return "ACK Flood"
    return None

def send_email(subject, message):
    try:
        from_email = 'idssolutions123@gmail.com'
        password = 'yyjwyodyxyyzcyii'
        to_email = 'alexande-cs18002@stu.kln.ac.lk'

        msg = MIMEMultipart()
        msg['From'] = from_email
        msg['To'] = to_email
        msg['Subject'] = subject
        msg['Date'] = formatdate(localtime=True)

        msg.attach(MIMEText(message, 'plain'))

        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(from_email, password)
        server.sendmail(from_email, to_email, msg.as_string())
        server.quit()

        print("Email sent successfully.")
    except Exception as e:
        print(f"Failed to send email: {e}")

def predict_attack():
    table = table_var.get()
    record_id = id_entry.get()

    if not table or not record_id:
        messagebox.showerror("Input Error", "Please select a table and enter a record ID.")
        return

    log_info, ip_address = read_data_in_db(table, record_id)
    if log_info is None:
        messagebox.showerror("Data Error", "No data found for the given ID.")
        return

    preprocessed_text = preprocess_text(log_info)
    new_text_bow = cv.transform([preprocessed_text]).toarray()
    predicted_class = classifier.predict(new_text_bow)

    if predicted_class[0] == 1:
        attack_type = determine_attack_type(log_info)
        if attack_type:
            email_subject = "Network Anomaly Detection"
            email_message = f"""
            An attack has been detected from the IP address: {ip_address}.

            Record ID: {record_id}
            
            Log Information:
            {log_info}
            
            Type of Attack: {attack_type}

            Risk Profile: High
            """
            send_email(email_subject, email_message)
            messagebox.showinfo("Prediction Result", f"Flood attack detected.\nPredicted Class: 1")
        else:
            messagebox.showinfo("Prediction Result", "No relevant attack detected.")
    else:
        messagebox.showinfo("Prediction Result", f"No attack detected.\nPredicted Class: 0")

# Creating the GUI
root = tk.Tk()
root.title("Network Anomaly Detection")

# Instruction label
instruction_label = tk.Label(root, text="Please select the table and the ID from network_data database")
instruction_label.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

# Dropdown for table selection
tk.Label(root, text="Select Table:").grid(row=1, column=0, padx=10, pady=10)
table_var = tk.StringVar()
table_dropdown = ttk.Combobox(root, textvariable=table_var)
table_dropdown['values'] = ('net_queries_syn', 'net_queries_ack', 'net_queries_normal')
table_dropdown.grid(row=1, column=1, padx=10, pady=10)

# Entry for record ID
tk.Label(root, text="Enter Record ID:").grid(row=2, column=0, padx=10, pady=10)
id_entry = tk.Entry(root)
id_entry.grid(row=2, column=1, padx=10, pady=10)

# Predict button
predict_button = tk.Button(root, text="Predict", command=predict_attack)
predict_button.grid(row=3, column=0, columnspan=2, pady=20)

root.mainloop()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dilki\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
